## **Question Answering (QA) in Natural Language Processing (NLP)**

Question Answering (QA) is a subfield of NLP focused on automatically providing precise answers to questions posed in natural language. QA systems aim to understand the context and meaning of questions and retrieve or generate accurate responses. 

**Keys Concepts**

1. **Types of Question Answering Systems**
    - Extractive QA: These systems identify and extract the answer from a given context or document. For example, given a passage of text, the system finds the segment that directly answers the question.
    - Abstractive QA: These systems generate answers based on the context without strictly extracting text. They can paraphrase or summarize information to provide a coherent response.

2. **Techniques Used in QA**
    - Information Retrieval: This approach involves searching for relevant documents or snippets in a large corpus based on the question. The answers are then extracted from these documents.
    - Text Understanding: Natural language understanding (NLU) techniques are employed to interpret the question's semantics, including named entity recognition (NER) and part-of-speech tagging.
    - Machine Learning: Deep learning models, such as transformers (e.g., BERT, GPT, RoBERTa), are often utilized for both extractive and abstractive QA tasks. They are trained on large datasets to understand context and generate human-like responses.

## **🤗 Pre-trained model**: [`roberta-base for Extractive QA`](https://huggingface.co/deepset/roberta-base-squad2)

The `deepset/roberta-base-squad2` model uses the RoBERTa (_Robustly Optimized BERT Approach_) architecture, a variation of BERT that has been optimized to offer better results in text comprehension tasks and it was fine-tuned using the SQuAD2.0 dataset. It's been trained on question-answer pairs, including unanswerable questions, for the task of Extractive Question Answering. In the context of QA, this means that the model can analyze a block of text and answer questions formulated in natural language about that text.  

In [5]:
from transformers import pipeline

QA_model = pipeline('question-answering', model='deepset/roberta-base-squad2')

**How QA Model works?**

The model receives two crucial pieces of information: the question and the context. The question is formulated by the user and seeks specific information, while the context is the text or document that contains the data needed to formulate an answer.

The first step in manipulating this data is tokenization, where the text is broken down into smaller units, such as words. Each of these tokens is then transformed into a numeric vector through a process called embedding, which captures the semantic meaning and usage of each word.

After the initial preparation, question and context are often concatenated into a single sequence with a special token, such as [SEP] in BERT-based models, demarcating the transition between question and context. This concatenated text is then submitted to the attention model, a central feature of transformer architectures, which allows the system to evaluate the relevance of each word in the context of the question. Attention helps the model focus on specific parts of the text that are more likely to contain the answer.

The innovation of attention models is that they don't just analyze each word individually, but also consider the relationship between all the words in the sequence. This allows the model to understand the text in a more holistic and contextualized way, which is crucial for answering questions accurately. Based on this contextual understanding, the model then predicts probabilities for each position in the text, indicating where the answer might begin and end.

This ability to process and answer questions autonomously and accurately makes the QA engine extremely valuable for a variety of applications, from virtual assistants to enterprise solutions for knowledge management and customer support, facilitating quick access to accurate information without the need for extensive manual searching and reading of documents.

In [6]:
def get_answer(question, context):
  return QA_model(question=question, context=context)

In [7]:
# Example of how to use this model
question = 'Why is model conversion important?'
context = 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'

In [8]:
get_answer(question, context)

{'score': 0.21171429753303528,
 'start': 59,
 'end': 84,
 'answer': 'gives freedom to the user'}

In [9]:
question = 'What are the payment options available?'
context = 'In our e-commerce, you can make payments using credit card, debit card, Apple Pay, or through bank transfers.'

get_answer(question, context)

{'score': 0.817491352558136,
 'start': 47,
 'end': 108,
 'answer': 'credit card, debit card, Apple Pay, or through bank transfers'}

In [10]:
# Create a Frequently Asked Questions - FAQ
contexts = {
    "How do I create an account?": "You can create an account by clicking on the 'Sign Up' button on our homepage. You will need to provide your email address, create a password, and fill in some basic information about yourself. An account will help you track your orders, manage your personal settings, and speed up future transactions.",
    "Which payment methods do you accept?": "We accept a wide range of payment methods including Visa, MasterCard, American Express, Discover, PayPal, and Apple Pay. You can also pay using store credit or gift cards issued by our company.",
    "How can I track my order?": "Once your order has shipped, you will receive an email with a tracking number. You can use this number on our website's tracking page to see the current status of your delivery.",
    "Do you offer international shipping?": "Yes, we offer international shipping to most countries. Shipping costs and delivery times vary depending on the destination. All applicable customs fees, taxes, and duties are the responsibility of the customer and are calculated at checkout.",
    "How long does delivery take?": "For standard shipping, deliveries typically take between 3 to 5 business days. For expedited shipping, expect your order to arrive within 1 to 2 business days. Delivery times may vary based on your location and the time of the year.",
    "What is your return policy?": "Our return policy allows you to return products within 30 days of receiving them. Items must be in their original condition and packaging. Some items, such as perishable goods, are not eligible for return.",
    "Can I change or cancel my order after it's been placed?": "You can change or cancel your order within 24 hours of placing it without any additional charge.To make changes or cancel your order, please contact our customer service immediately.",
    "What should I do if I receive a damaged item?": "If you receive a damaged item, please contact our customer service within 48 hours of delivery to report the damage. You will need to provide your order number, the description of the damage, and photographic evidence. We will arrange for a replacement or refund as appropriate.",
    "How do I reset my password?": "If you've forgotten your password, go to the login page and click on 'Forgot Password'. Enter your email address and we will send you a link to reset your password. For security purposes, this link will expire within 24 hours."
}

In [11]:
def faq_answer(question):
  context = contexts[question]
  result = QA_model(question=question, context=context)
  return result['answer']

faq_answer('How do I create an account?')

"by clicking on the 'Sign Up' button on our homepage"

In [15]:
faq_answer('How long does delivery take?')

'between 3 to 5 business days'

In [18]:
# it should be the same answer before, but it will result in KeyError
try:
    answer = faq_answer('What is the delivery time?')
except KeyError:
    print("This question is not present in the context passed.")

This question is not present in the context passed.


- To handle situations where the input question does not exactly match the keys in your FAQ dictionary, you can use similarity metrics to find the closest match. 

1. **Use a Similarity Measure**: it's possible to use similarity measure after use a vector representation of the questions.
2. **Preprocess the Questions**: before calculating similarity, it's necessary to preprocess both the input question and the keys in the context dictionary (e.g., lowercasing, removing punctuation).

In [21]:
# Using similarity-based approach using scikit-learn and tfidfvectorizer to compute cosine similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def new_faq_answer(question):
    # Prepare the data
    faq_questions = list(contexts.keys())
    
    # Create the vectorizer and fit_transform the questions
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(faq_questions + [question])

    # Compute the cosine similarity
    similarity_matrix = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    
    # Get the index of the most similar question
    most_similar_index = np.argmax(similarity_matrix)
    
    # Retrieve the closest question
    closest_question = faq_questions[most_similar_index]
    
    # Get the context for the closest question
    context = contexts[closest_question]
    
    # Here you should call your QA model with the context
    result = QA_model(question=closest_question, context=context)
    return result['answer']

In [29]:
new_faq_answer('standard delivery time?')

'between 3 to 5 business days'

In [23]:
new_faq_answer('How long delivery')

'between 3 to 5 business days'

In [24]:
new_faq_answer('want to create an account')

"by clicking on the 'Sign Up' button on our homepage"

In [25]:
new_faq_answer('available payment methods')

'Visa, MasterCard, American Express, Discover, PayPal, and Apple Pay'